In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes
!pip install xformers

* We support Llama, Mistral, CodeLlama, TinyLlama, Vicuna, Open Hermes etc
* And Yi, Qwen ([llamafied](https://huggingface.co/models?sort=trending&search=qwen+llama)), Deepseek, all Llama, Mistral derived archs.
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] With [PR 26037](https://github.com/huggingface/transformers/pull/26037), we support downloading 4bit models **4x faster**! [Our repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.25.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/198k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.21G [00:00<?, ?B/s]

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `ChatML` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [ ]:


from datasets import load_dataset

dataset = load_dataset("codeparrot/xlcost-text-to-code", "C++-program-level", split ="train")

In [ ]:
dataset

In [ ]:
print(dataset[:5])  # Shows the first 5 samples

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["text"]
    outputs       = examples["code"]

    texts = []
    for instruction, output in zip(instructions, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [ ]:

# from datasets import load_dataset
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 10,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Maximum Prefix Sum possible by merging two given arrays | C ++ Program to implement the above approach ; Stores the maximum prefix sum of the array A [ ] ; Traverse the array A [ ] ; Stores the maximum prefix sum of the array B [ ] ; Traverse the array B [ ]", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 5000, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
pip install nltk


In [ ]:
import torch
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformers import AutoTokenizer, AutoModelForCausalLM

# Ensure nltk resources are downloaded
nltk.download('punkt')

# Initialize BLEU score variables
total_bleu_score = 0
num_samples = 10  # You want to iterate over the first 20 samples

# Iterate over the first 20 instructions
for i in range(num_samples):
    instruction = dataset[i]["text"]  # Replace with correct field if needed
    reference_code = dataset[i]["code"]  # Replace with correct field if needed

    # Prepare input for the model
    inputs = tokenizer(
        [
            alpaca_prompt.format(instruction, "")
        ], return_tensors="pt").to("cuda")

    # Generate output using the model
    outputs = model.generate(**inputs, max_new_tokens=5000, use_cache=True)
    tokenizer.batch_decode(outputs)
    generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Tokenize the reference and generated code
    reference_tokens = [nltk.word_tokenize(reference_code)]
    generated_tokens = nltk.word_tokenize(generated_code)

    # Compute BLEU score
    smoothing_function = SmoothingFunction().method1
    bleu_score = sentence_bleu(reference_tokens, generated_tokens, smoothing_function=smoothing_function)

    # Accumulate BLEU score
    total_bleu_score += bleu_score

# Compute average BLEU score over all 20 samples
average_bleu_score = total_bleu_score / num_samples

print(f"Average BLEU Score for the first {num_samples} samples: {average_bleu_score:.4f}")


In [ ]:
# import torch
# import nltk
# from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
# from transformers import AutoTokenizer, AutoModelForCausalLM

# # Ensure nltk resources are downloaded
# nltk.download('punkt')

# # Initialize BLEU score variables
# total_bleu_score = 0
# num_samples = 20  # You want to iterate over the first 20 samples

# # Iterate over the first 20 instructions
# for i in range(num_samples):
#     instruction = dataset[i]["text"]  # Replace with correct field if needed
#     reference_code = dataset[i]["code"]  # Replace with correct field if needed

#     # Prepare input for the model
#     inputs = tokenizer(
#         [
#             alpaca_prompt.format(instruction, "")
#         ], return_tensors="pt").to("cuda")

#     # Generate output using the model
#     outputs = model.generate(**inputs, max_new_tokens=5000, use_cache=True)
#     generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

#     # Tokenize the reference and generated code
#     reference_tokens = [nltk.word_tokenize(reference_code)]
#     generated_tokens = nltk.word_tokenize(generated_code)

#     # Compute BLEU score
#     smoothing_function = SmoothingFunction().method1
#     bleu_score = sentence_bleu(reference_tokens, generated_tokens, smoothing_function=smoothing_function)

#     # Accumulate BLEU score
#     total_bleu_score += bleu_score

# # Compute average BLEU score over all 20 samples
# average_bleu_score = total_bleu_score / num_samples

# print(f"Average BLEU Score for the first {num_samples} samples: {average_bleu_score:.4f}")
